In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import os
import state_code
import collections

In [2]:
from state_code import CODE_R

In [2]:
source = requests.get('https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports/')

soup = bs(source.text,'html.parser')
page = soup.prettify()

In [3]:
web_fileset = set()
web_set = set()
for i in soup.find_all('a', class_='js-navigation-open'):
    if 'csv' in i['href']:
        web_set.add(i['href'])
        web_fileset.add(i['href'].split('/')[-1])

dir_set = set(os.listdir('./dataset'))
new_data = web_fileset.difference(dir_set)
for i in new_data:
    path = 'https://raw.githubusercontent.com'+''.join(i.split('/blob'))
    # raw_source = requests.get(path)
    # raw_soup = bs(raw_source.text, 'html.parser')
    # csv_path = 'https://raw.githubusercontent.com'+raw_soup.find(id='raw-url')['href']
    print(path)
    df = pd.read_csv(path)
    filename = i.split('/')[-1]
    #print(filename)
    df.to_csv('./dataset/'+filename)


In [3]:
files = os.listdir('./dataset/')
files.sort()

In [7]:
df = pd.read_csv('./dataset/04-21-2020.csv')
res = df['Province_State'] == 'Illinois'
check = set()
print(len(df[res]))
for i in df[res]['Combined_Key']:
    check.add(i)
print(len(check))

output_df = []
for i in files:
    #print(i)
    total = {}
    raw_df = pd.read_csv('./dataset/'+i)
    country = [col for col in raw_df.columns if 'Country' in col][0]
    state = [col for col in raw_df.columns if 'Province' in col][0]
    check = raw_df[country] == 'US'
    df = raw_df[check]

    for _, row in df.iterrows():
        us_state = row[state].split(',')[0]
        if us_state not in state_code.CODE:
            if len(row[state].split(',')) < 2 or len((row[state].split(',')[1]).split(' ')[1]) < 2:
                continue
            s_code = (row[state].split(',')[1]).split(' ')[1]
            if s_code in state_code.CODE_R:
                us_state = state_code.CODE_R[s_code]
            else:
                print(row[state])
                continue
        #print(row[state],i)
        if us_state in total:
            total[us_state][0] += row['Confirmed']
            total[us_state][1] += row['Deaths']
            total[us_state][2] += row['Recovered']
            #total[row[state]][3] += row['Active']
        else:
            total[us_state] = [row['Confirmed'],row['Deaths'],row['Recovered']]
    #print(total)
    usa_df = pd.DataFrame(columns=['code','total_confirmed','total_death','total_recovered','date'])
    for j in total:
        a_row = [state_code.CODE[j]]+total[j]+[i[:10]]
        leng = len(usa_df)
        usa_df.loc[leng] = a_row
        #row_df = pd.DataFrame([a_row])
        # usa_df = usa_df.append(a_row, ignore_index=True)
    #print(usa_df)
    output_df.append(usa_df.fillna(0))

97
97


In [13]:
a = ', jiang '
(a.split(',')[1]).split(' ')

['', 'jiang', '']

In [14]:
import plotly.graph_objects as go
import numpy as np

# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for i in output_df:
    fig.add_trace(
        go.Choropleth(
    locations=i['code'], # Spatial coordinates
    z = i['total_death'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "People",
    text=usa_df['total_death']
            ))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 91},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title_text = 'COVID-19 USA Data Visualization',
    geo_scope='usa', # limite map scope to USA
)

fig.show()